<a href="https://colab.research.google.com/github/BastiaanGrisel/TK2021/blob/main/Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install adjustText

In [11]:
import pandas as pd

import altair as alt
import matplotlib.pyplot as plt

import seaborn as sns
import matplotlib.colors as mc

from sklearn.decomposition import PCA

# Data Opschonen

In [25]:
df = pd.read_csv("besluiten.csv", usecols=["fractie", "besluit", "stem"])
df

,fractie,besluit,stem
0,DENK,8c0cdf15-b2f7-49e3-b2cf-f74127b3e59f,Voor
1,50PLUS,dabf87bd-1c0c-4da4-a95a-6bce227598c3,Voor
2,ChristenUnie,3def8cb0-dd9c-4218-8655-dd879c29eba8,Tegen
3,GroenLinks,095a8ea0-5358-40fe-b89f-c80a98a66968,Tegen
4,Van Kooten-Arissen,ad446f7b-a4d2-415d-85f5-f2eafd3f514e,Voor
...,...,...,...
232280,SGP,6fbf0f92-fa7a-435c-8cae-63bf8489fc68,Voor
232281,D66,9d150040-e274-43cb-b3e1-71d4ce8e8055,Voor
232282,Houwers,47dd3f67-287b-4d42-9f40-5820d16cb1e5,Tegen
232283,CDA,ee50a3a9-e6a0-4ab2-82cb-51497d0389eb,Tegen


In [27]:
# Tellen hoeveel besluiten het zijn
len(df["besluit"].unique())

14798

In [13]:
# Alleen stemmen voor of tegen worden meegenomen (er staat ook "Niet deelgenomen". Aanname is dat er drie categorien stemmen zijn: voor, tegen, onbekend)
df = df[df["stem"].isin(["Voor", "Tegen"])]

In [14]:
# Pak per stemming één stem per fractie (aanname: iedereen binnen een fractie stemt hetzelfde)
df = df.groupby(["fractie", "besluit"])["stem"].first().reset_index()
df

,fractie,besluit,stem
0,50PLUS,000b7778-f15a-461e-8a69-22a71d923125,Tegen
1,50PLUS,000ee695-e74a-445e-b3a2-9cc3f9bc0cd6,Voor
2,50PLUS,00103441-703c-414b-ae62-ff0d5eae1d88,Voor
3,50PLUS,00110985-76fe-4963-8ad4-ae26eabcda0f,Tegen
4,50PLUS,001d06fe-cd7c-47e2-a84f-8d83a098a806,Tegen
...,...,...,...
208580,Van Vliet,fcc9414d-f40c-44cf-b413-2aff3cb22bde,Tegen
208581,Van Vliet,fd73f7be-73c1-489d-8a74-68746939c680,Voor
208582,Van Vliet,fdf79238-d1f5-4f98-a9a0-0b70c52ea16d,Voor
208583,Van Vliet,ff113d7c-f44f-4c07-8cc6-7c12bf68393a,Voor


In [15]:
# Over fracties die weinig gestemd hebben is niet genoeg informatie 
# dan correleren ze met elkaar omdat ze weinig gestemd hebben niet omdat ze iets inhoudelijks met elkaar te maken hebben
fracties = df.groupby("fractie")[["stem"]].count().query("stem > 10000").reset_index()["fractie"]

df = df_source = df[df['fractie'].isin(fracties)]

# Overgebleven fracties
fracties

0           50PLUS
1              CDA
2     ChristenUnie
3              D66
4             DENK
5              FVD
6       GroenLinks
7              PVV
8             PvdA
9             PvdD
10             SGP
11              SP
12             VVD
Name: fractie, dtype: object

In [16]:
# Maak voor elke fractie een rij met wat ze gestemd hebben
df = df.pivot(index="fractie", columns="besluit", values="stem")

In [22]:
# Gebruik one-hot encoding om Voor/Tegen/Onbekend uit te splitsen
df = pd.get_dummies(df)
df

,000b7778-f15a-461e-8a69-22a71d923125_Tegen,000b7778-f15a-461e-8a69-22a71d923125_Voor,000ee695-e74a-445e-b3a2-9cc3f9bc0cd6_Tegen,000ee695-e74a-445e-b3a2-9cc3f9bc0cd6_Voor,00103441-703c-414b-ae62-ff0d5eae1d88_Tegen,00103441-703c-414b-ae62-ff0d5eae1d88_Voor,00110985-76fe-4963-8ad4-ae26eabcda0f_Tegen,00110985-76fe-4963-8ad4-ae26eabcda0f_Voor,001d06fe-cd7c-47e2-a84f-8d83a098a806_Tegen,001d06fe-cd7c-47e2-a84f-8d83a098a806_Voor,001ec9cd-6388-492e-bf91-9d397af5bbeb_Tegen,001ec9cd-6388-492e-bf91-9d397af5bbeb_Voor,002b498f-ce4b-4316-a38e-af9fa1ab422c_Tegen,002b498f-ce4b-4316-a38e-af9fa1ab422c_Voor,00304454-9a24-451e-8480-8eec3980fe6b_Tegen,00304454-9a24-451e-8480-8eec3980fe6b_Voor,003becea-de90-47bc-9b42-9815a2badf25_Tegen,003becea-de90-47bc-9b42-9815a2badf25_Voor,003e4eab-02d5-4968-a7ca-1f70a0e60386_Tegen,003e4eab-02d5-4968-a7ca-1f70a0e60386_Voor,003ebc15-2787-4327-8385-c9448df792b8_Tegen,003ebc15-2787-4327-8385-c9448df792b8_Voor,004123b5-82e2-4874-899a-ab523c4f50e4_Tegen,004123b5-82e2-4874-899a-ab523c4f50e4_Voor,004b70f2-cf0d-4827-8002-31e10929d3db_Voor,005089af-d2ce-466c-a601-8cbb1d164e53_Voor,0057e1a3-d5a1-4f9a-9822-fdd2a8e4ad5c_Tegen,0057e1a3-d5a1-4f9a-9822-fdd2a8e4ad5c_Voor,0058340d-a819-439c-bea3-3156a366aaf3_Tegen,0058340d-a819-439c-bea3-3156a366aaf3_Voor,00616580-b5da-45d6-b159-2d2723a1ad1d_Voor,0066510b-02a8-43c8-9b99-a2854720887a_Tegen,0066510b-02a8-43c8-9b99-a2854720887a_Voor,0067f630-5dfc-4905-8e95-e00d893bfee9_Tegen,0067f630-5dfc-4905-8e95-e00d893bfee9_Voor,006cb37d-5e44-4af7-947c-3df72923bf6a_Tegen,006cb37d-5e44-4af7-947c-3df72923bf6a_Voor,006d12b3-ce61-458c-a342-5dab55f0d03d_Tegen,006d12b3-ce61-458c-a342-5dab55f0d03d_Voor,0073bd18-47a6-479d-bf12-9b9957389106_Voor,...,ff7cabea-a65e-4fbb-b5ff-d4b52832b515_Voor,ff867f6c-7ad5-47a0-88fc-ce6d9b42c8d4_Tegen,ff867f6c-7ad5-47a0-88fc-ce6d9b42c8d4_Voor,ff8799f1-59d2-43a6-8a93-5a4f6ddf3ffb_Tegen,ff8799f1-59d2-43a6-8a93-5a4f6ddf3ffb_Voor,ff9faef2-32b4-48b6-9a17-2b0a4cf81571_Tegen,ff9faef2-32b4-48b6-9a17-2b0a4cf81571_Voor,ff9fe7d3-2993-4d5c-b07e-3ac221b13a82_Tegen,ff9fe7d3-2993-4d5c-b07e-3ac221b13a82_Voor,ffa09bb8-d0d8-46da-8e66-8172c4c3bdd5_Tegen,ffa09bb8-d0d8-46da-8e66-8172c4c3bdd5_Voor,ffae9a93-f77c-4fa7-805d-2f5b9448f4a8_Tegen,ffae9a93-f77c-4fa7-805d-2f5b9448f4a8_Voor,ffb171e2-f2be-4c45-be77-911856387892_Tegen,ffb171e2-f2be-4c45-be77-911856387892_Voor,ffb3896d-df00-49f7-90c0-4dd983c91ae0_Tegen,ffb3896d-df00-49f7-90c0-4dd983c91ae0_Voor,ffb69f1e-5d2e-45b1-86b4-824615088c45_Tegen,ffb69f1e-5d2e-45b1-86b4-824615088c45_Voor,ffb8ea25-4f34-4d88-b78c-f4adc4ed8344_Tegen,ffb8ea25-4f34-4d88-b78c-f4adc4ed8344_Voor,ffbe688a-1051-49db-a1ff-da48b99fbd46_Tegen,ffbe688a-1051-49db-a1ff-da48b99fbd46_Voor,ffc1854d-bc85-4bed-ae6e-a4fafc3b0749_Tegen,ffc1854d-bc85-4bed-ae6e-a4fafc3b0749_Voor,ffc52a14-3b52-4efa-adc6-1cc1f44e8584_Tegen,ffc52a14-3b52-4efa-adc6-1cc1f44e8584_Voor,ffc5e197-aad2-48c2-8bc5-137ffaec1563_Tegen,ffc5e197-aad2-48c2-8bc5-137ffaec1563_Voor,ffd5ae3f-441e-4adb-ad5e-723b2b5ec3df_Tegen,ffd5ae3f-441e-4adb-ad5e-723b2b5ec3df_Voor,ffda10d1-e27c-4606-830d-600e685e9fe6_Tegen,ffda10d1-e27c-4606-830d-600e685e9fe6_Voor,ffe3d572-8095-4404-942c-2fb1b0ed61b1_Tegen,ffe3d572-8095-4404-942c-2fb1b0ed61b1_Voor,ffec6852-575c-4d4f-a6b3-ced8a7ac3bc1_Tegen,ffec6852-575c-4d4f-a6b3-ced8a7ac3bc1_Voor,fff8f065-ced7-400e-9044-379b91ac2821_Tegen,fff8f065-ced7-400e-9044-379b91ac2821_Voor,fffdf76a-8ab1-439e-b9ec-8ebbeb8e53c9_Voor
fractie,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
50PLUS,1,0,0,1,0,1,1,0,1,0,1,0,1,0,0,1,0,1,0,1,0,1,1,0,1,1,0,1,1,0,1,1,0,0,1,0,1,0,1,1,...,1,0,1,0,1,1,0,0,1,0,1,0,1,0,1,1,0,1,0,0,1,0,1,1,0,1,0,0,1,0,1,0,1,1,0,0,1,1,0,1
CDA,1,0,0,1,0,1,0,1,1,0,1,0,1,0,1,0,1,0,0,1,1,0,1,0,1,1,1,0,1,0,1,1,0,0,1,1,0,0,1,1,...,1,1,0,0,1,1,0,0,1,1,0,1,0,1,0,1,0,1,0,0,1,0,1,1,0,1,0,1,0,1,0,0,1,1,0,0,1,1,0,1
ChristenUnie,1,0,0,1,0,1,0,1,1,0,1,0,1,0,1,0,1,0,0,1,1,0,1,0,1,1,1,0,1,0,1,1,0,0,1,1,0,0,1,1,...,1,1,0,0,1,1,0,0,1,1,0,1,0,0,1,1,0,1,0,0,1,0,1,1,0,1,0,1,0,1,0,0,1,1,0,0,1

# Correlatiematrix met Partijen

In [18]:
# Bereken de correlatie matrix met alle partijen
dfc = df.T.corr()
dfc.index.rename("fractie2", inplace=True)
dfc = dfc.reset_index().melt(id_vars="fractie2")

alt.Chart(dfc).mark_circle().encode(
    x = alt.X('fractie', axis=alt.Axis(title=None)),
    y = alt.Y('fractie2', axis=alt.Axis(title=None), sort = "x"),
    size = alt.Size('value:Q', legend=None),
    color = alt.Color('value:Q', scale=alt.Scale(scheme='redblue')),
    tooltip = 'value'
).properties(title='Correlatie tussen stemgedrag van partijen').interactive()

alt.Chart(...)

# Politiek Landschap

In [20]:
# Bereken de correlatiematrix en voer daar PCA op uit
pca = PCA(n_components = 2)
pca_emb = pca.fit_transform(df.T.corr())

df_pca = pd.DataFrame(pca_emb, columns = ["x", "y"])
df_pca["fractie"] = df.index.values

pca.explained_variance_ratio_.sum()

0.9079361035940748

In [21]:
# Plaats de labels zodat ze niet overlappen
df_pca["labx"] = df_pca["x"]
df_pca["laby"] = df_pca["y"]

df_pca.loc[df_pca["fractie"] == "GroenLinks", ["labx", "laby"]] += [-0.2, 0]
df_pca.loc[df_pca["fractie"] == "PvdA", ["labx", "laby"]] += [0.1, 0]
df_pca.loc[df_pca["fractie"] == "PvdD", ["labx", "laby"]] += [0, 0.18]
df_pca.loc[df_pca["fractie"] == "D66", ["labx", "laby"]] += [-0.05, 0]
df_pca.loc[df_pca["fractie"] == "ChristenUnie", ["labx", "laby"]] += [0.15, 0]

# Gebruik de kleurtjes van de partijen (beetje desaturated anders is het alemaal zo fel)
color_scale = alt.Scale(
            domain = ['50PLUS', 'CDA', 'ChristenUnie', 'D66', 'DENK', 'FVD', 'GroenLinks', 'PVV', 'PvdA', 'PvdD', 'SGP', 'SP', 'VVD'],
            range = [mc.to_hex(sns.desaturate(x, 0.6)) for x in ['#650757', '#007b5f', "#072563", "#00ae41", "#00b6b1", "#841a1a", "#7ec345", "#004473", "#e1091c", "#006b2c", "#e95d0f", "#ed1c24", "#ff6400"]],
            type = "ordinal")

# Plot de text
alt_labels = alt.Chart(df_pca).mark_text(align='center', dy=12).encode(
    alt.X('labx', axis=None),
    alt.Y('laby', axis=None),
    text ="fractie",
    color = alt.Color('fractie', scale = color_scale, legend = None)
)

# Plot de punten
alt_points = alt.Chart(df_pca).mark_point(size = 60).encode(
    x='x',
    y='y',
    text ="fractie",
    color = alt.Color('fractie', scale = color_scale, legend = None)
)

# Plot de tekst en de punten in één plaatje
(alt_labels + alt_points).configure_view(strokeOpacity=0)

alt.LayerChart(...)